# Optical mark recognition

### Importing the libraries

In [70]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2

### Initializing the answer key

In [71]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

### Reading the image

In [97]:
image = cv2.imread('omr_test_03.png')

### Converting it to grayscale

In [98]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

### Bluring it to hide the details

In [99]:
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

### Performing edge detection

In [100]:
edged = cv2.Canny(blurred, 75, 200)

### Displaying the image

In [37]:
cv2.imshow('img',edged)
cv2.waitKey(0)

-1

### Finding the contours

In [101]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [102]:
cnts = cnts[1]

### Determining contour of the paper

In [103]:
docCnt = None
if len(cnts) > 0:
     # Sorting contours based on their area. Paper will have mamimum area as it is the main object
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)  
    
    for c in cnts:
        
        peri = cv2.arcLength(c, True)   # Calculating the perimeter
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)    # approximating the shape
        
        if len(approx) == 4:    # If shape has four edges
            docCnt = approx
            break

### Visualizing the contours

In [41]:
cv2.imshow('temp',cv2.drawContours(image.copy(),[docCnt],-1,(0,0,255),2))
cv2.waitKey(0)

-1

### Applying four point transformation

In [104]:
paper = four_point_transform(image,docCnt.reshape(4,2))

In [105]:
warped = four_point_transform(gray,docCnt.reshape(4,2))

### Visualizing the transformed image

In [44]:
cv2.imshow('img',paper)
cv2.waitKey(0)

-1

### Thresholding

In [106]:
# OTSU will automatically determine the best value for the parameter thresh. The value of thresh is not considered 
# when otsu flag is passed
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]   
cv2.imshow("Thresh", thresh)                                    
cv2.waitKey(0)

-1

### Finding the contours of the thresholded image

In [107]:
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[1]

### Determining the contours of the bubble

In [108]:
questionCnts = []
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)     # Computing the bounding box for the bubble 
    ar = w / float(h)     # Computing the aspect ratio of the bounding box
    
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

In [109]:
len(questionCnts)

25

### Visualizing the contour of bubbles

In [49]:
pap = paper.copy()
for c in questionCnts:
    cv2.imshow('temp',cv2.drawContours(pap,[c],-1,(0,0,255),2))
    cv2.waitKey(0)

### Sorting the contours from top to bottom

In [110]:
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]

### Visualizing the sorted contour of bubbles

In [86]:
pap = paper.copy()
for c in questionCnts:
    cv2.imshow('temp',cv2.drawContours(pap,[c],-1,(0,255,0),2))
    cv2.waitKey(0)

### Isolating the bubbles

In [89]:
pap = paper.copy()
q = -1
correct = 0
minm = []
maxm = []
for i in range(0,len(questionCnts),5):
    q+=1
    
    # Sorting contours from left to right
    temp = contours.sort_contours(questionCnts[i:i+5], method="left-to-right")[0]
    
    bubbled = None
    for (j,c) in enumerate(temp):
        
        # Refer to last section ('Isolation step by step') for viewing what each step does 
        
        mask = np.zeros(thresh.shape, dtype="uint8")     # Create a dummy mask
        cv2.drawContours(mask, [c], -1, 255, -1)         # Drawing the contours on the dummy mask
                                                                       
        
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)     # Doing Bitwise-and to reveal the bubble
        
        total = cv2.countNonZero(mask)     # Calculating the sum of non-zero pixels
        minm.append(total)     # For use in future
        if bubbled is None or total > bubbled[0]:     # Comparing the sum of non zero pixels
            bubbled = (total, j)
    
    maxm.append(bubbled[0])     # For use in future
    
    color = (0, 0, 255)
    k = ANSWER_KEY[q]     # Retrieving the answer from the answer key based on question 'q'
    
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
    cv2.drawContours(pap, [temp[k]], -1, color, 3)
        

### Displaying the score

In [92]:
score = (correct /(q+1)) * 100
cv2.putText(pap, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("test", pap)
cv2.waitKey(0)

-1

###  

# But there's a problem
### Till now we had user who would only mark one bubble  for each question. What if : 
1. No bubble is marked
2. A nefarious user marks all the bubbles 

Both the above problems can be solved using a small logic

1. Using a minimum value of total for the bubble to be marked
2. We can set a flag if an option has been marked breforehand

In [93]:
print(maxm)

[817, 686, 794, 805, 722]


We can see that most of the marked bubble have <u> value of total greater than 600 </u>

In [94]:
print(minm)

[289, 817, 290, 320, 300, 279, 328, 290, 307, 686, 794, 318, 285, 326, 316, 294, 323, 805, 331, 320, 286, 722, 294, 324, 317]


And for the unmarked bubble the <u> values of total is less than 600 <u> 

### Modified code

In [111]:
pap = paper.copy()
q = -1
correct = 0
flag = False
minm = []
maxm = []
for i in range(0,len(questionCnts),5):
    q+=1
    
    # Sorting contours from left to right
    temp = contours.sort_contours(questionCnts[i:i+5], method="left-to-right")[0]
    
    bubbled = None
    flag = False
    for (j,c) in enumerate(temp):
        
        mask = np.zeros(thresh.shape, dtype="uint8")     # Create a dummy mask
        cv2.drawContours(mask, [c], -1, 255, -1)         # Drawing the contours on the dummy mask
                                                                       
        
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)     # Doing Bitwise-and to reveal the bubble
        
        total = cv2.countNonZero(mask)     # Calculating the sum of non-zero pixels            
        
        # Breaking the loop and setting bubbled = None when another marked bubble is found in same question
        if flag and total > 600:     
            bubbled = None
            break
            
        if total > 600:     # Setting flag = True when a marked bubble is found
            flag = True
            
        if total > 600:     # Comparing the sum of non zero pixels
            bubbled = (total, j)
                
    color = (0, 0, 255)
    k = ANSWER_KEY[q]     # Retrieving the answer from the answer key based on question 'q'
    
    if bubbled and k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
    if bubbled:     # Not Drawing contour when bubbled is None
        cv2.drawContours(pap, [temp[k]], -1, color, 3)

In [115]:
score = (correct /(q+1)) * 100
cv2.putText(pap, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("test", pap)
cv2.waitKey(0)

-1

### Isolation step by step

In [116]:
mask = np.zeros(thresh.shape, dtype="uint8")     # Create a dummy mask

In [117]:
cv2.imshow('temp',mask)
cv2.waitKey(0)

-1

In [118]:
c = contours.sort_contours(questionCnts[0:5], method="left-to-right")[0]     # Sorting contours from left to right

In [119]:
temp = cv2.drawContours(mask, [c[0]], -1, 255, -1)     # Drawing the contours on the dummy mask

In [120]:
cv2.imshow('temp',temp)
cv2.waitKey(0)

-1

In [121]:
mask = cv2.bitwise_and(thresh, thresh, mask=mask)     # Doing Bitwise-and to reveal the bubble

In [122]:
cv2.imshow('temp',mask)
cv2.waitKey(0)

-1